In [170]:
import yfinance as yf
data1 = yf.download("^NDX ^NSEI", group_by='Ticker', start="2010-01-01", end="2023-05-01")

[*********************100%***********************]  2 of 2 completed


In [171]:
#Dropping the NaN values from the dataframe
data2=data1.dropna()
data2.insert(0,"Number", [*range(0, len(data2), 1)])

In [172]:
df=data2.drop([('^NSEI','Volume'),('^NSEI','Adj Close'),('^NDX','Volume'),('^NDX','Adj Close')],axis=1)

In [173]:
for i in ("^NSEI", "^NDX"):
 df[i,"ATR"]=(df[i,"High"]-df[i,"Low"])

In [174]:
corr_coeff_0 = df['^NSEI','Close'].corr(df['^NDX','Close'])
corr_coeff_1 = df['^NSEI','Close'].corr(df['^NDX','Close'].shift(20))
corr_coeff_2 = df['^NDX','Close'].corr(df['^NSEI','Close'].shift(20))
print(corr_coeff_0)
print(corr_coeff_1)
print(corr_coeff_2)

0.9548767284940565
0.9554576673977387
0.9457068004332322


In [175]:
#the correlation coeffiecient between the closing of NSE and NASDAQ indexes is 0.954877 which implies a very strong linear
#relationship.The value is very close to 1 which implies that usually increase or decrease in price of one of them is
#accompanied by the simultaneous increase or decrease of the other.

In [176]:
#shifting NASDAQ by 20 days shows a better correlation coeffiecient than earlier. It implies that NSE lags NASDAQ by 20 days

In [177]:
df['^NDX','daily_returns'] = 1+df['^NDX','Close'].pct_change()
df['^NSEI','daily_returns'] = 1+df['^NSEI','Close'].pct_change()

In [178]:
import pandas as pd
dates = pd.DataFrame(columns = ["Buy_date", "Sell_date", "Buy_price", "Sell_price","returns","buydays","selldays"])

In [179]:
def MACD_signals(df,index):
    Short_EMA = df[index,'Close'].ewm(span=11, adjust=False).mean()
    Long_EMA = df[index,'Close'].ewm(span=30, adjust=False).mean()
    MACD = Short_EMA - Long_EMA
    signal = MACD.ewm(span=6, adjust=False).mean()
    buy_signal= MACD>signal
    sell_signal= MACD<signal
    return buy_signal,sell_signal
#the optimized parameters are 11,30,6 days

In [180]:
def bollinger_signals(df,index):
    Bollinger_Middle=df[index,'Close'].rolling(20).mean()
    Bollinger_Upper=Bollinger_Middle+2*df[index,'Close'].rolling(35).std()
    Bollinger_Lower=Bollinger_Middle-2*df[index,'Close'].rolling(35).std()
    buy_signal=df[index,'Close'] < Bollinger_Lower
    sell_signal=df[index,'Close'] > Bollinger_Upper
    return buy_signal,sell_signal
#the optimized parameters are multiplier=2,20 days, 35 days

In [181]:
def keltner_signals(df,index):
    Keltner_Middle=df[index,'Close'].ewm(span=19, adjust=False).mean()
    Keltner_Upper=Keltner_Middle+2.1*df[index,'ATR'].rolling(18).mean()
    Keltner_Lower=Keltner_Middle-2.1*df[index,'ATR'].rolling(18).mean()
    buy_signal=df[index,'Close'] < Keltner_Lower
    sell_signal=df[index,'Close'] > Keltner_Upper
    return buy_signal,sell_signal
#the optimized parameters are multiplier=2.1,19 days, 18 days

In [182]:
def calculate(df,index,buy_signal,sell_signal,dates):
    buying_dates=[]
    selling_dates=[]
    Buy_price=[]
    Sell_price=[]
    t=0
    for i in df.index:
        if(t==0):
            if(buy_signal[i]==True):
                buying_dates.append(i)
                Buy_price.append(df[index,'Close'][i])
                t=1
                continue
        if(t==1):
            if(sell_signal[i]==True):
                selling_dates.append(i)
                Sell_price.append(df[index,'Close'][i])
                t=0
    dates= dates[0:0]
    if(len(buying_dates)-len(selling_dates)!=0):
        buying_dates.pop()
        Buy_price.pop()
    dates.loc[:,'Sell_date']=selling_dates
    dates.loc[:,'Buy_date']=buying_dates
    dates.loc[:,'Buy_price']=Buy_price
    dates.loc[:,'Sell_price']=Sell_price
    dates.loc[:,'returns']=1+(dates['Sell_price']-dates['Buy_price'])/dates['Buy_price']
    for i in dates.index:
        dates.loc[i,'buydays']=df['Number'][dates['Buy_date'][i]]
        dates.loc[i,'selldays']=df['Number'][dates['Sell_date'][i]]
        if(i==0):
            new=df[index,'daily_returns'].iloc[dates['buydays'][0]+1:dates['selldays'][0]+1]
            newclose=df[index,'Close'].iloc[dates['buydays'][0]:dates['selldays'][0]+1]
        else:
            new=new._append(df[index,'daily_returns'].iloc[dates['buydays'][i]+1:dates['selldays'][i]+1])
            newclose=newclose._append(df[index,'Close'].iloc[dates['buydays'][i]:dates['selldays'][i]+1])
    CummulativeReturn=(dates['returns'].product())
    averageReturn=CummulativeReturn**(252/len(new))
    sharperatio=((averageReturn-1)*100-5.25)/(((new-1)*100).std()*(252**0.5))
    mdd=0
    while i<len(newclose):
        current_mdd=(newclose[i]-newclose[i+1:].min())/newclose[i]
        if(current_mdd>mdd):
            mdd=current_mdd
        i=i+1
    mdd=-mdd
    CummulativeReturn=(CummulativeReturn-1)*100
    mdd=mdd*100
    return CummulativeReturn,sharperatio,mdd,dates

In [183]:
buy,sell = MACD_signals(df,'^NDX')
cr,sr,mdd,dates = calculate(df,'^NDX',buy,sell,dates)
print(cr)
print(sr)
print(mdd)

272.1134724967453
0.951086798032722
-35.47246350158443


In [184]:
buy,sell = bollinger_signals(df,'^NDX')
cr,sr,mdd,dates = calculate(df,'^NDX',buy,sell,dates)
print(cr)
print(sr)
print(mdd)

734.8038795653025
1.0961209030596082
-35.54042647502641


In [185]:
buy,sell = keltner_signals(df,'^NDX')
cr,sr,mdd,dates = calculate(df,'^NDX',buy,sell,dates)
print(cr)
print(sr)
print(mdd)

375.3177910672504
1.2528070516213703
-30.701874842725967


In [186]:
buy,sell = MACD_signals(df,'^NSEI')
cr,sr,mdd,dates = calculate(df,'^NSEI',buy,sell,dates)
print(cr)
print(sr)
print(mdd)
print(dates)

198.3227636011096
0.9079364920518446
-34.60925610603369
                Buy_date  Sell_date     Buy_price    Sell_price   returns   
0    2010-01-05 00:00:00 2010-01-12   5277.899902   5210.399902  0.987211  \
1    2010-01-15 00:00:00 2010-01-19   5252.200195   5225.649902  0.994945   
2    2010-02-16 00:00:00 2010-04-12       4855.75   5339.700195  1.099665   
3    2010-05-28 00:00:00 2010-07-01   5066.549805   5251.399902  1.036484   
4    2010-07-12 00:00:00 2010-07-20        5383.0        5368.0  0.997213   
..                   ...        ...           ...           ...       ...   
134  2022-10-14 00:00:00 2022-11-21  17185.699219  18159.949219   1.05669   
135  2022-11-28 00:00:00 2022-12-06      18562.75      18642.75   1.00431   
136  2023-01-18 00:00:00 2023-01-27  18165.349609  17604.349609  0.969117   
137  2023-02-08 00:00:00 2023-02-22  17871.699219  17554.300781   0.98224   
138  2023-03-06 00:00:00 2023-03-13  17711.449219  17154.300781  0.968543   

    buydays sellday

In [187]:
buy,sell = bollinger_signals(df,'^NSEI')
cr,sr,mdd,dates = calculate(df,'^NSEI',buy,sell,dates)
print(cr)
print(sr)
print(mdd)
print(dates)

123.80716585571622
0.4647968136868895
-37.627037683360584
               Buy_date  Sell_date     Buy_price    Sell_price   returns   
0   2010-05-04 00:00:00 2010-06-21        5148.5   5353.299805  1.039779  \
1   2010-11-19 00:00:00 2011-03-30   5890.299805   5787.649902  0.982573   
2   2011-06-20 00:00:00 2011-07-07   5257.899902   5728.950195  1.089589   
3   2011-08-05 00:00:00 2011-10-28       5211.25   5360.700195  1.028678   
4   2011-11-21 00:00:00 2012-09-17   4778.350098        5610.0  1.174045   
5   2012-11-16 00:00:00 2012-11-29   5574.049805        5825.0  1.045021   
6   2013-02-26 00:00:00 2013-09-19   5761.350098   6115.549805  1.061479   
7   2014-02-03 00:00:00 2014-03-06   6001.799805   6401.149902  1.066538   
8   2014-12-16 00:00:00 2015-01-20   8067.600098   8695.599609  1.077842   
9   2015-03-26 00:00:00 2016-05-26   8342.150391   8069.649902  0.967335   
10  2016-11-11 00:00:00 2017-01-11   8296.299805   8380.650391  1.010167   
11  2017-09-27 00:00:00 2019-0

In [188]:
buy,sell = keltner_signals(df,'^NSEI')
cr,sr,mdd,dates = calculate(df,'^NSEI',buy,sell,dates)
print(cr)
print(sr)
print(mdd)
print(dates)

111.839994840299
0.5237372834699371
-37.627037683360584
               Buy_date  Sell_date     Buy_price    Sell_price   returns   
0   2010-01-29 00:00:00 2010-03-16   4882.049805   5198.100098  1.064737  \
1   2010-05-07 00:00:00 2010-06-21   5018.049805   5353.299805  1.066809   
2   2010-11-19 00:00:00 2011-03-28   5890.299805       5687.25  0.965528   
3   2011-05-04 00:00:00 2011-06-30   5537.149902   5647.399902  1.019911   
4   2011-08-04 00:00:00 2011-10-28   5331.799805   5360.700195   1.00542   
5   2011-11-17 00:00:00 2012-01-17       4934.75   4967.299805  1.006596   
6   2012-05-08 00:00:00 2012-06-29   4999.950195   5278.899902   1.05579   
7   2012-07-26 00:00:00 2012-08-07        5043.0   5336.700195  1.058239   
8   2013-02-26 00:00:00 2013-04-25   5761.350098   5916.299805  1.026895   
9   2013-06-11 00:00:00 2013-07-12   5788.799805        6009.0  1.038039   
10  2013-07-30 00:00:00 2013-09-10   5755.049805       5896.75  1.024622   
11  2014-01-30 00:00:00 2014-02-